In [146]:
import csv
import pandas as pd
#import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import string
from nltk.tokenize import RegexpTokenizer
from itertools import chain
import json

In [147]:
#read csv as data frame with pandas
df = pd.read_csv("Airbnb_Texas_Rentals.csv")

#remove \\n new line from the description
df = df.replace(r'\\n', " ", regex = True)

#make the df as list
df1 = df.values.tolist()

#create a tsv file with \t tab space for each row 
for row in df1:
    with open("doc_tsv/doc_" +str(df1.index(row))+ ".tsv", 'w', newline='',encoding='utf8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t',)
        tsv_output.writerow(row[1:])

In [148]:
#remove the index
for i in df1:
    i.remove(i[0])

In [149]:
#download the stop words
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [150]:
stopwords = stopwords.words('english') # get english stop words



In [151]:
#we tokenize except for the symbol $ that remains with the value
tokenizer = RegexpTokenizer("[\w'\$]+")


In [152]:
#stemming
st = PorterStemmer()

In [153]:
#tokenize description
df.description = df.description.apply(str).apply(tokenizer.tokenize)


In [154]:
#tokenize title
df.title = df.title.apply(str).apply(tokenizer.tokenize)


In [155]:
#remove stop word and punctuation from description and title
df.description = df.description.apply(lambda x : [item for item in x if item not in stopwords])

In [156]:
df.title = df.title.apply(lambda x : [item for item in x if item not in stopwords])

In [157]:
#remove stemmings from descr and title
df.description = df.description.apply(lambda x : [st.stem(y) for y in x])
 

In [158]:
df.title = df.title.apply(lambda x : [st.stem(y) for y in x])

# 3.1 Conjunctive query

### 3.1.1 Create your index!

In [159]:
voc = []

for i in list(df.description):
    for j in i:
        voc.append(j)
for i in list(df.title):
    for j in i:
        voc.append(j)
        
voc = set(voc) #all distinct words


In [160]:
num = list(voc)
vocabulary = {}

for i in num:
    vocabulary.update({num.index(i):i})

In [161]:
#vocabulary

In [162]:
with open("vocabulary.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(vocabulary))

In [163]:
#create dict from df that assign for each rows all the words in
newdict = {}
for index, row in df.iterrows():
    newdict["doc_" + str(index)] = list(row["description"]) + list(row["title"])

In [164]:
with open("index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(newdict))


In [165]:
#create inverted index called inverse

inverse = dict() 
for key in newdict: 
    # Go through the list that is saved in the dict:
    for item in set(newdict[key]):
        # Check if in the inverted dict the key exists
        if item not in inverse: 
            # If not create a new list
            inverse[item] = [key] 
        else: 
            inverse[item].append(key) 


In [166]:
# change the keys of inverse with the keys of vocabulary
for i in vocabulary:
    inverse[i] = inverse[vocabulary[i]]
    del inverse[vocabulary[i]]

In [167]:
#save inverted index as json
with open("inverted_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverse))

In [168]:
query=input() #user input
query=tokenizer.tokenize(query)
query=[t for t in query if t not in stopwords]
query=[st.stem(word) for word in query]
index = json.loads(open('inverted_index.json').read())
vocabulary = json.loads(open('vocabulary.json').read())
vocabulary={k:v for v,k in vocabulary.items()}
for i in set(query):
    if i in vocabulary:
        query.remove(i)
        query.append(vocabulary[i])
    else:
        query.remove(i)
docs=[]
query=set(query)
for i in query:
    docs.append(set(index[i]))
docs=set.intersection(*docs)
results=[]
for file in docs:
    opentsv=open("doc_tsv/"+file+".tsv","r",encoding="utf8")
    lines=opentsv.readlines()
    for x in lines:
        results.append([x.split('\t')[7],x.split('\t')[4],x.split('\t')[2],x.split('\t')[8]])
    opentsv.close()
result=pd.DataFrame(results,columns=['Title','Description','City','URL'])
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)
result.style.set_properties(*{'text-align': 'center'})
result.style.format({'URL': make_clickable})

garden beach


,Title,Description,City,URL
0,Shep's Shack,"Shep's Shack is only 8 blocks from the Beach. You’ll love the home because of the neighborhood & surrounding opportunities for fun (i.e., Moody Gardens & Pleasure Pier.) The neighborhood includes a private boat slip for bay fishing, a community pool & a basketball court that are only 4 blocks away. The house also has washers, cornhole, horseshoes, darts & foosball. It is good for all sorts of travelers from families to big groups of friends. It fits at least 9 comfortably & pets are welcomed.",Jamaica Beach,https://www.airbnb.com/rooms/14918295?location=Bayou%20Vista%2C%20TX
1,"Restful, quiet stay in Santa Fe Texas.","My place is close to Kemah Boardwalk, Galveston beach, Moody Gardens several nice places within close driving distance. You’ll love my place because of the comfy king size bed, the coziness, the kitchen, the views, quiet location, closeness to all type of businesses and activities. My place is good for couples, solo adventurers, and business Galveston, NASA, Kemah etc. then back to enjoy peace and tranquility! Central to many medical facilities. \",Santa Fe,https://www.airbnb.com/rooms/13466814?location=Bayou%20Vista%2C%20TX
2,Classic Galveston Beach Bungalow,"Our cool and cozy downstairs unit is a great place to relax after a day on the beach or time spent enjoying local attractions. With the beach, Pleasure Pier, Kempner Park, and many restaurants and shops all within walking distance, you'll be close enough to enjoy all that Galveston has to offer. The outdoor patio and garden is a perfect place to unwind and enjoy cooking out on the BBQ pit. If you'd prefer Galveston's night life, there are also several bars and entertainment venues within walking distance. A perfect place for your Galveston getaway.",Galveston,https://www.airbnb.com/rooms/15753408?location=Bayou%20Vista%2C%20TX
3,Sailboat Retreat,"Beautiful Hunter Sailboat that will provide you with an incomparable on the water experience. Perfect for couples, adventurers, small families and business travelers. Docked at South Shore Harbor Resort which is close to the Kemah Boardwalk, NASA Space Center, Moody Gardens and Galveston beach. The resort has an awesome pool with swim up bar, restaurant and laundry center.",League City,https://www.airbnb.com/rooms/19454515?location=Baytown%2C%20TX
4,Home near Galveston Beach and other attractions,"Nice 4 Bedroom 2 Bath home in a great location close to freeways. 5 min to Tanger Outlet Mall 15 min from Galveston Beach/Schlitterbahn/Moody Gardens / Texas City Dike 18 minutes to NASA 23 min Kema Boardwalk 35 min Houston Hobby Airport 50 min to Houston (Minute Maid Park, NRG Stadium, Zoo and Musems 60 min to George Bush Intercontinel Airport",Texas City,https://www.airbnb.com/rooms/18076465?location=Bayou%20Vista%2C%20TX
5,Beach Front Condo- KING Bed. Sleeps Six,"Welcome to our Condo. Perfect location for some relaxing beach time. Walk across the street to the Beach. Sleeps six with a Gulf view balcony, living area with sleeper sofa, kitchen, master bedroom with KING bed, twin bunks, and full bath. TWO pools, one heated and one kid friendly pool. Two hot tubs, Wi-Fi, covered parking, workout facility, and restaurant. Close to retailers and grocery stores. Minutes from the Strand, Moody Gardens, Schlitterbahn, and Pleasure Pier. We love our guest.",Galveston,https://www.airbnb.com/rooms/17386776?location=Bayou%20Vista%2C%20TX
6,Nirhanah By The Sea,"2006 Construction. 3 Br., 2 Ba., full kitchen. Balcony and Master with ocean view. Travertine floors, granite counter tops, high ceilings, jetted tub and bidet in both bathrooms. Exercise room and two pools (kid pool and lap pool.) Sleeps up to 8. Two assigned parking spaces. Convenient to public beaches, fishing pier, Schlitterbahn water park, Moody Gardens and downtown Galveston. Walking distance to fishing pier, restaurants and putt-putt golf. 8 guest maximum, please.",Galveston,https://www.airbnb.com/rooms/18525223?location=Bayo

In [169]:
vocablist=[x for x in vocabulary.keys()]

In [170]:

corpus=json.loads(open('index.json').read())


In [171]:
vocabulary

{'efficieni': '0',
 'sentiment': '1',
 'lunch': '2',
 '501': '3',
 'veeeri': '4',
 'hottub': '5',
 'volunt': '6',
 'htx': '7',
 'instant': '8',
 'aggrav': '9',
 'evs': '10',
 'docksid': '11',
 "macy'": '12',
 'bubbl': '13',
 'keyboard': '14',
 '저희': '15',
 'name': '16',
 "10'": '17',
 'cloud': '18',
 'dust': '19',
 'awe': '20',
 'eiffel': '21',
 'medow': '22',
 'eilan': '23',
 'thermador': '24',
 '1942': '25',
 'wingstop': '26',
 'thorough': '27',
 'jacinto': '28',
 'contemporarili': '29',
 'beneath': '30',
 'rugged': '31',
 'peden': '32',
 'carport': '33',
 'taylor': '34',
 'stranger': '35',
 'ibm': '36',
 'headach': '37',
 'galaxi': '38',
 'constant': '39',
 '3bedroom': '40',
 'convien': '41',
 'AQ': '42',
 'brisco': '43',
 'greenhol': '44',
 'peel': '45',
 'inland': '46',
 'C': '47',
 'whi': '48',
 'symphoni': '49',
 'healthcar': '50',
 'et': '51',
 'unfenc': '52',
 'avoid': '53',
 'stuffi': '54',
 'project': '55',
 "corgi'": '56',
 'underway': '57',
 'airplan': '58',
 "randall'": '

In [172]:
index

{'0': ['doc_9625'],
 '1': ['doc_11415', 'doc_15633'],
 '2': ['doc_610',
  'doc_1818',
  'doc_1960',
  'doc_2145',
  'doc_2470',
  'doc_2605',
  'doc_2695',
  'doc_5192',
  'doc_5385',
  'doc_5589',
  'doc_6187',
  'doc_6197',
  'doc_7088',
  'doc_8286',
  'doc_8484',
  'doc_10102',
  'doc_11167',
  'doc_11866',
  'doc_11957',
  'doc_12123',
  'doc_12155',
  'doc_12345',
  'doc_12909',
  'doc_12965',
  'doc_12998',
  'doc_15028',
  'doc_15560',
  'doc_15988',
  'doc_16309',
  'doc_17528',
  'doc_18039'],
 '3': ['doc_3754', 'doc_5503', 'doc_7980', 'doc_10327', 'doc_15290'],
 '4': ['doc_459'],
 '5': ['doc_25',
  'doc_814',
  'doc_1247',
  'doc_4102',
  'doc_4932',
  'doc_6458',
  'doc_6963',
  'doc_8846',
  'doc_8936',
  'doc_9074',
  'doc_9368',
  'doc_9657',
  'doc_9821',
  'doc_10913',
  'doc_11421',
  'doc_11620',
  'doc_11990',
  'doc_12341',
  'doc_12365',
  'doc_15598',
  'doc_15955',
  'doc_16774',
  'doc_17327'],
 '6': ['doc_1760',
  'doc_2110',
  'doc_2223',
  'doc_2630',
  'doc

In [173]:
inverted_index=json.loads(open('inverted_index.json').read())
vocab=json.loads(open('vocabulary.json').read())

In [174]:
idf={}
for term in inverted_index:
    term1=vocab[term]
    idf[term]=math.log(1+(len(corpus)/len(term1)))
with open("idf_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverted_index))

In [175]:
import math
for term in inverted_index:
    term1=vocab[term]
    for doc in inverted_index[term]:
        count=(corpus[doc].count(term1)/len(corpus[doc]))*idf[term]
        inverted_index[term][inverted_index[term].index(doc)]=(doc,count)
with open("tfidf_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverted_index))

In [180]:
k=5
query=input()
error=[]
query=tokenizer.tokenize(query)
query=[t for t in query if t not in stopwords]
query=[st.stem(word) for word in query]
for i in query:
    if i in vocabulary:
        query[query.index(i)]=vocabulary[i]
    else:
        error.append(i)
        query.remove(i)
if error !=[]:
    print('there are no results that match your search criteria: '+', '.join(error))
else:
    query_tfidf={}
    for i in query:
        query_tfidf[i]=(query.count(i)/len(query))*(idf[i])
    
    result_docs=[]
    for i in query:
        docs=[]
        for j in inverted_index[i]:
            docs.append(j[0])
        result_docs.append(set(docs))
    result_docs=list(set.intersection(*result_docs))
    docsidf={}
    for i in query:
        docsidf[i]=[item for item in inverted_index[i] if item[0] in result_docs]

    print(docsidf)
    


garden beach
{'1576': [('doc_1344', 0.15727416607845743), ('doc_1673', 0.19097577309526972), ('doc_1874', 0.23591124911768613), ('doc_1929', 0.14071899070177768), ('doc_2243', 0.2765856024138389), ('doc_3122', 0.22917092771432368), ('doc_6071', 0.15727416607845743), ('doc_6775', 0.20052456175003322), ('doc_9086', 0.13149151590166114), ('doc_11272', 0.20052456175003322), ('doc_13168', 0.1293706850000214), ('doc_13380', 0.1293706850000214), ('doc_13558', 0.12339973030771276), ('doc_13601', 0.1293706850000214), ('doc_13839', 0.15133929188681752), ('doc_13847', 0.1336830411666888), ('doc_14448', 0.15727416607845743), ('doc_14589', 0.15727416607845743), ('doc_14925', 0.40104912350006644), ('doc_14961', 0.12532785109377076), ('doc_15117', 0.20052456175003322), ('doc_15150', 0.2864636596429046), ('doc_15206', 0.3084993257692819), ('doc_15227', 0.12339973030771276), ('doc_15286', 0.20052456175003322), ('doc_15456', 0.20052456175003322), ('doc_15568', 0.11971615626867654), ('doc_15610', 0.15424